<font face="B Mitra" size=4>
<div dir=rtl align=center>
<br>
<img src="https://aut.ac.ir/templates/tmpl_modern01/images/logo_fa.png" alt="Amirkabir University Logo" width="100">
<br>
<font size=5> دانشگاه صنعتی امیرکبیر
(پلی تکنیک تهران)
<br>
<font size=5> دانشکده ریاضی و علوم کامپیوتر
<br>
<font size=6>
<b>پروژه ۲ درس مباحثی در ریاضیات و کاربردها</b>
<br>
<font size=6>
<b>تشخیص دست نوشته با استفاده از تجزیه QR</b>
<br>
<font size=5> پیاده سازی: روژان واصف - محمدعرفان سلیمی
<br>
<font size=5> استاد درس:فاطمه شاکری
<br>
<font size=4> فروردین ۱۴۰۴
<hr>
</div>
</font>

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
بارگذاری کتابخانه های مورد نیاز


In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
 بارگذاری مجموعه داده
EMINIST

In [26]:
# Load the EMNIST Letters Dataset
def load_emnist():
    transform = torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(),  # Convert to grayscale
        torchvision.transforms.ToTensor()  # Convert to PyTorch tensor
    ])

    # Load training and test datasets
    train_dataset = torchvision.datasets.EMNIST(root="./data", split="letters", train=True, download=True, transform=transform)
    test_dataset = torchvision.datasets.EMNIST(root="./data", split="letters", train=False, download=True, transform=transform)

    return train_dataset, test_dataset

train_dataset, test_dataset = load_emnist()

# Convert to NumPy arrays
X_train = np.array([train_dataset[i][0].numpy().flatten() for i in range(len(train_dataset))])
y_train = np.array([train_dataset[i][1] for i in range(len(train_dataset))])

X_test = np.array([test_dataset[i][0].numpy().flatten() for i in range(len(test_dataset))])
y_test = np.array([test_dataset[i][1] for i in range(len(test_dataset))])

# Normalize pixel values (0 to 1 range)
X_train /= 255.0
X_test /= 255.0

# Adjust labels to start from 0 (because EMNIST labels go from 1 to 26)
y_train -= 1
y_test -= 1

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
انتخاب ۲۰۰ نمونه به عنوان داده های آموزشی
و
۲۰ نمونه به عنوان داده های تست
از هر کلاس

In [27]:
# Define samples per class
num_train_samples = 200
num_trainـadded_samples = 20
num_test_samples = 20
num_classes = 26  # a-z

# Create training and testing matrices for each letter
train_matrices = []
train_added_matrices = []
test_matrices = []

for i in range(num_classes):
    train_indices = np.where(y_train == i)[0][:num_train_samples]
    train_added_indices = np.where(y_train == i)[0][num_train_samples: num_train_samples + num_trainـadded_samples]
    test_indices = np.where(y_test == i)[0][:num_test_samples]

    train_matrices.append(X_train[train_indices])
    train_added_matrices.append(X_train[train_added_indices])
    test_matrices.append(X_test[test_indices])

# Convert lists to NumPy arrays
train_matrices = np.array(train_matrices)
test_matrices = np.array(test_matrices)
train_added_matrices = np.array(train_added_matrices)

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
محاسبه تجزیه QR ماتریس ها با استفاده از ماتریس های هاوس هولدر

In [28]:
def householder_matrix(a):
    n = len(a)
    denominator = a[0] + np.copysign(np.linalg.norm(a), a[0])
    if denominator == 0:
        return np.eye(n)
    v = a / denominator
    v[0] = 1
    Q = np.eye(n) - (2 / (v @ v)) * (v[:, None] @ v[None, :])
    return Q

def qr_decomposition_householder(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()

    for j in range(min(m - 1, n)):
        a = R[j:, j]
        Q_j = householder_matrix(a)
        R[j:, j:] = Q_j @ R[j:, j:]
        Q_temp = np.eye(m)
        Q_temp[j:, j:] = Q_j
        Q = Q @ Q_temp

    return Q, R

# Compute QR decompositions for each class (transpose needed)
Q_matrices = []
R_matrices = []

for i in range(num_classes):
    A = train_matrices[i].T
    Q, R = qr_decomposition_householder(A)
    Q_matrices.append(Q)
    R_matrices.append(R)

# Prediction function using reconstruction error
def predict_class_1(y):
    errors = []
    for i in range(num_classes):
        Q_i = Q_matrices[i]
        R_i = R_matrices[i]

        b_i = Q_i.T @ y
        x_i = np.linalg.lstsq(R_i, b_i, rcond=None)[0]
        y_hat_i = Q_i @ (R_i @ x_i)
        error_i = np.linalg.norm(y - y_hat_i)
        errors.append(error_i)

    return np.argmin(errors)

# Evaluate model on test data
correct = 0
total_samples = 0

for i in range(num_classes):
    for j in range(num_test_samples):
        y_test = test_matrices[i][j].reshape(-1, 1)
        predicted = predict_class_1(y_test)

        if predicted == i:
            correct += 1
        total_samples += 1

accuracy = (correct / total_samples) * 100
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 70.58%


<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
محاسبه تجزیه QR ماتریس ها با استفاده از ماتریس های گیونز

In [29]:
def givens_rotation(a, b):
    if b != 0:
        denom = np.hypot(a, b)
        c = a / denom
        s = -b / denom
    else:
        c = 1.0
        s = 0.0
    return c, s

def update_qr_givens(Q, R, new_column):
    m, n = R.shape
    new_R = np.hstack([R, new_column.reshape(-1, 1)])
    new_Q = Q.copy()

    for i in range(m - 1):
        c, s = givens_rotation(new_R[i, -1], new_R[i+1, -1])
        G = np.array([[c, -s], [s, c]])

        new_R[[i, i+1], :] = G @ new_R[[i, i+1], :]
        new_Q[:, [i, i+1]] = new_Q[:, [i, i+1]] @ G.T

    return new_Q, new_R

for i in range(num_classes):
    new_data = train_added_matrices[i].T

    for j in range(20):
        new_col = new_data[:, j]

        Q_matrices[i], R_matrices[i] = update_qr_givens(Q_matrices[i], R_matrices[i], new_col)

# Prediction
def predict_class_2(y):
    errors = []
    for i in range(num_classes):
        Q_i = Q_matrices[i]
        R_i = R_matrices[i]
        b_i = Q_i.T @ y
        x_i = np.linalg.lstsq(R_i, b_i, rcond=None)[0]
        y_hat_i = Q_i @ (R_i @ x_i)
        error_i = np.linalg.norm(y - y_hat_i)
        errors.append(error_i)
    return np.argmin(errors)

# Accuracy test
correct = 0
total_samples = 0

for i in range(num_classes):
    for j in range(20):
        y_test = test_matrices[i][j].reshape(-1, 1)
        predicted_class = predict_class_2(y_test)
        if predicted_class == i:
            correct += 1
        total_samples += 1

accuracy = (correct / total_samples) * 100
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 70.19%
